# **Logistic Regression**

Logistic Regression adalah metode statistik yang digunakan untuk memprediksi hasil dari variabel dependen biner (dua kelas) berdasarkan satu atau lebih variabel independen. Tujuan dari logistic regression adalah untuk memodelkan probabilitas suatu peristiwa terjadi dengan memetakan input linier dari variabel-variabel independen menjadi rentang antara 0 dan 1 menggunakan fungsi logistik (sigmoid).

Dalam logistic regression, kurva yang dihasilkan bukanlah garis lurus, melainkan kurva berbentuk S (sigmoid). Model ini memperkirakan probabilitas suatu kelas, di mana probabilitas lebih besar dari 0,5 biasanya dikategorikan sebagai satu kelas, dan kurang dari 0,5 sebagai kelas lainnya. Logistic regression sering digunakan dalam kasus seperti prediksi.

#### **persamaan Logistic Regression**

$$ P(Y = 1 | X) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n)}} $$

Di mana:
- \( P(Y = 1 | X) \) adalah probabilitas bahwa kelas output \( Y \) adalah 1, diberikan fitur \( X \).
- \( \beta_0 \) adalah intercept (bias), dan \( \beta_1, \beta_2, \dots, \beta_n \) adalah koefisien atau bobot fitur \( X_1, X_2, \dots, X_n \).
- \( e \) adalah bilangan Euler, dan fungsi logistik memastikan bahwa output berada dalam rentang 0 hingga 1.


#### **fungsi sigmoid**

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Di mana $ \sigma(z) $ adalah fungsi sigmoid, dan $z$ adalah input linier yang dihasilkan dari variabel independen model.


#### **Library**

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import re
#stopwords
import nltk
from nltk.corpus import stopwords

#stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

#spit data
from sklearn.model_selection import train_test_split

#### **Load Data**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/PPW/tugas

/content/drive/MyDrive/PPW/tugas


In [ ]:
df = pd.read_csv("Tugas-Crawling-Data-Berita-2-kategori.csv")
df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik
...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup


#### **Preprosesing**

##### **1. Mengubah kolom kategori menjadi bentuk biner**

In [ ]:
df['Kategori'] = df['Kategori Berita'].map({'Politik': 0, 'Gaya Hidup': 1})

##### **2. Cleansing**

In [ ]:
def remove_url(data_berita):
  url = re.compile(r'https?://\S+|www\.S+')
  return url.sub(r'', data_berita)

def remove_html(data_berita):
  html = re.compile(r'<.#?>')
  return html.sub(r'', data_berita)

def remove_emoji(data_berita):
  emoji_pattern = re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF""]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', data_berita)

def remove_numbers(data_berita):
  data_berita = re.sub(r'\d+', '', data_berita)
  return data_berita

def remove_symbols(data_berita):
  data_berita = re.sub(r'[^a-zA-Z0-9\s]', '', data_berita)
  return data_berita

df['cleansing'] = df['Isi Berita'].apply(lambda x: remove_url(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_html(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_emoji(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_symbols(x))
df['cleansing'] = df['cleansing'].apply(lambda x: remove_numbers(x))

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...
...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu komunitas ...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...


##### **3. Case folding**

In [ ]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['cleansing'].apply(case_folding)

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...
...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu komunitas ...,timesindonesia malang pada minggu komunitas ...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...


##### **4. Tokenization**

In [ ]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding,tokenize
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi..."
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan..."
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ..."
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb..."
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,..."
...,...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg..."
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu komunitas ...,timesindonesia malang pada minggu komunitas ...,"[timesindonesia, malang, pada, minggu, komunit..."
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,..."
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic..."


##### **5. Stop Words**

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))

df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding,tokenize,stopword_removal
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...
...,...,...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu komunitas ...,timesindonesia malang pada minggu komunitas ...,"[timesindonesia, malang, pada, minggu, komunit...",timesindonesia malang minggu komunitas oom ir ...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...


##### **6. Stemming**

In [ ]:
# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi stemming
def stemming(text):
    return [stemmer.stem(word) for word in text]

# Terapkan stemming setelah stopwords removal
df['stemming'] = df['stopword_removal'].apply(lambda x: ' '.join(stemming(x.split())))

df

,NO,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita,Kategori,cleansing,case_folding,tokenize,stopword_removal,stemming
0,1,"Analisis PPI: Di Tangan Presiden Prabowo, Indo...","TIMESINDONESIA, JAKARTA – Di bawah kepemimpina...",04/09/2024 - 23:35,Politik,0,TIMESINDONESIA JAKARTA Di bawah kepemimpinan ...,timesindonesia jakarta di bawah kepemimpinan ...,"[timesindonesia, jakarta, di, bawah, kepemimpi...",timesindonesia jakarta kepemimpinan presiden r...,timesindonesia jakarta pimpin presiden ri prab...
1,2,"Bacawabup Lathifah Solidkan Seribu Relawan, Si...","TIMESINDONESIA, MALANG – Bakal cawabup Malang,...",04/09/2024 - 22:02,Politik,0,TIMESINDONESIA MALANG Bakal cawabup Malang La...,timesindonesia malang bakal cawabup malang la...,"[timesindonesia, malang, bakal, cawabup, malan...",timesindonesia malang cawabup malang lathifah ...,timesindonesia malang cawabup malang lathifah ...
2,3,Rois Syuriah Sebut Visi Misi Untoro-Wahyudi Se...,"TIMESINDONESIA, BANTUL – Rois Syuriah PCNU Ban...",04/09/2024 - 20:54,Politik,0,TIMESINDONESIA BANTUL Rois Syuriah PCNU Bantu...,timesindonesia bantul rois syuriah pcnu bantu...,"[timesindonesia, bantul, rois, syuriah, pcnu, ...",timesindonesia bantul rois syuriah pcnu bantul...,timesindonesia bantul rois syuriah pcnu bantul...
3,4,Ribuan Kader PKB Kabupaten Mojokerto Panasi Me...,"TIMESINDONESIA, MOJOKERTO – Ribuan kader PKB K...",04/09/2024 - 19:23,Politik,0,TIMESINDONESIA MOJOKERTO Ribuan kader PKB Kab...,timesindonesia mojokerto ribuan kader pkb kab...,"[timesindonesia, mojokerto, ribuan, kader, pkb...",timesindonesia mojokerto ribuan kader pkb kabu...,timesindonesia mojokerto ribu kader pkb kabupa...
4,5,"Wajah Pengurus Inti DPP PKB Tidak Berubah, Ini...","TIMESINDONESIA, JAKARTA – Partai Kebangkitan B...",04/09/2024 - 16:43,Politik,0,TIMESINDONESIA JAKARTA Partai Kebangkitan Ban...,timesindonesia jakarta partai kebangkitan ban...,"[timesindonesia, jakarta, partai, kebangkitan,...",timesindonesia jakarta partai kebangkitan bang...,timesindonesia jakarta partai bangkit bangsa p...
...,...,...,...,...,...,...,...,...,...,...,...
95,96,Azquira Scarf Luncurkan Hijab Autentik dengan ...,"TIMESINDONESIA, MALANG – Azquira Scarf menggel...",01/09/2024 - 22:42,Gaya Hidup,1,TIMESINDONESIA MALANG Azquira Scarf menggelar...,timesindonesia malang azquira scarf menggelar...,"[timesindonesia, malang, azquira, scarf, mengg...",timesindonesia malang azquira scarf menggelar ...,timesindonesia malang azquira scarf gelar acar...
96,97,Mengulik Sejarah Bangunan Heritage Kota Malang...,"TIMESINDONESIA, MALANG – Pada Minggu (1/9/2024...",01/09/2024 - 22:16,Gaya Hidup,1,TIMESINDONESIA MALANG Pada Minggu komunitas ...,timesindonesia malang pada minggu komunitas ...,"[timesindonesia, malang, pada, minggu, komunit...",timesindonesia malang minggu komunitas oom ir ...,timesindonesia malang minggu komunitas oom ir ...
97,98,"Ritual Wewangian, Parfum Lokal Jadi Pilihan In...","TIMESINDONESIA, SURABAYA – Memilih wewangian p...",31/08/2024 - 20:04,Gaya Hidup,1,TIMESINDONESIA SURABAYA Memilih wewangian pen...,timesindonesia surabaya memilih wewangian pen...,"[timesindonesia, surabaya, memilih, wewangian,...",timesindonesia surabaya memilih wewangian peng...,timesindonesia surabaya pilih wewangian harum ...
98,99,Berkeliling Nusantara Melalui Suguhan Kuliner ...,"TIMESINDONESIA, MAGELANG – Anda ingin mencicip...",04/09/2024 - 17:00,Gaya Hidup,1,TIMESINDONESIA MAGELANG Anda ingin mencicipi ...,timesindonesia magelang anda ingin mencicipi ...,"[timesindonesia, magelang, anda, ingin, mencic...",timesindonesia magelang mencicipi kenikmatan s...,timesindonesia magelang cicip nikmat deret kul...


##### **Split Data**

In [ ]:
x = df['stemming'].values
y = df['Kategori'].values

In [ ]:
Xtrain, Xtest,Ytrain,Ytest = train_test_split(x,y,test_size=0.2,random_state=2)

### **TF-IDF**

In [ ]:
vect = TfidfVectorizer()
X = vect.fit_transform(Xtrain)

X_array = vect.transform(Xtrain)

X_array.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0855273 ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.0855273 ],
       [0.09296982, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.11666782, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.010861  , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
with open('tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(vect, f)

### **Logistic Regression**

In [ ]:
# Membuat model Logistic Regression
model = LogisticRegression()

# Melatih model dengan data pelatihan
model.fit(X, Ytrain)

LogisticRegression()

In [ ]:
# Meyimpan model
with open('logistic_model', 'wb') as f:
    pickle.dump(model, f)

#### Pengujian menggunakan data tes

In [ ]:
# Load TF-IDF dan model logistic regression dari file pickle
with open("tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)

In [ ]:
with open("logistic_model", "rb") as f:
    model = pickle.load(f)

In [ ]:
# Transformasi data baru dengan TF-IDF yang sudah dilatih
X_new = tfidf_vectorizer.transform(Xtest)

# Lakukan prediksi dengan model logistic regression
prediksi = model.predict(X_new)

print("Prediksi:", prediksi)

Prediksi: [1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1]


In [ ]:
accuracy = accuracy_score(Ytest, prediksi)
print(f"Akurasi: {accuracy * 100:.2f}%")

Akurasi: 100.00%


### **Implementasi**

In [ ]:
# Load TF-IDF dan model logistic regression
with open('tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('logistic_model', 'rb') as f:
    logistic_model = pickle.load(f)

In [ ]:
# Load the TF-IDF vectorizer from the saved file
with open('tfidf_vectorizer.pkl', 'rb') as f:
    vect = pickle.load(f)

# Load the Logistic Regression model from the saved file
with open('logistic_model', 'rb') as f:
    model = pickle.load(f)

# Preprocessing functions

def clean_text(data_berita):
    """
    Cleans the input text by removing URLs, HTML tags, emojis, numbers, and symbols.
    """
    # Remove URLs
    data_berita = re.sub(r'https?://\S+|www\S+', '', data_berita)

    # Remove HTML tags
    data_berita = re.sub(r'<.*?>', '', data_berita)

    # Remove emojis
    data_berita = re.sub("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "]+", '', data_berita)

    # Remove numbers
    data_berita = re.sub(r'\d+', '', data_berita)

    # Remove symbols and keep only letters and spaces
    data_berita = re.sub(r'[^a-zA-Z\s]', '', data_berita)

    return data_berita

def case_folding(text):
    """
    Converts text to lowercase for normalization.
    """
    return text.lower()

def tokenize(text):
    """
    Splits text into individual tokens (words).
    """
    return text.split()

# Download stopwords for removing them
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('indonesian')

def remove_stopwords(text):
    """
    Removes common stopwords from the tokenized text.
    """
    return [word for word in text if word not in stop_words]

# Initialize the Indonesian stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(text):
    """
    Reduces words to their root forms using stemming.
    """
    return [stemmer.stem(word) for word in text]

# Function to preprocess the input text
def preprocess_input(text):
    """
    Complete preprocessing pipeline for the input news text, including URL, emoji,
    number removal, case folding, tokenization, stopwords removal, and stemming.
    """
    # Menggabungkan semua langkah pembersihan dalam satu fungsi
    text = clean_text(text)

    # Mengubah teks menjadi huruf kecil (case folding)
    text = case_folding(text)

    # Tokenisasi (memisahkan kata-kata)
    tokens = tokenize(text)

    # Menghapus stopwords
    tokens = remove_stopwords(tokens)

    # Stemming (mengembalikan kata ke bentuk dasarnya)
    stemmed_tokens = stemming(tokens)

    # Menggabungkan kembali kata yang sudah diolah menjadi string
    return ' '.join(stemmed_tokens)

# Function to predict category (Gaya Hidup or Politik)
def predict_category(text):
    """
    Takes preprocessed user input, transforms it using the saved TF-IDF vectorizer,
    and uses the trained Logistic Regression model to predict the category (0 for
    Politik, 1 for Gaya Hidup).
    """
    # Preprocess the input text
    preprocessed_text = preprocess_input(text)

    # Transform the preprocessed text using the loaded TF-IDF vectorizer
    tfidf_text = vect.transform([preprocessed_text])

    # Predict the category using the Logistic Regression model
    prediction = model.predict(tfidf_text)

    # Map the prediction to the corresponding category
    category = 'Politik' if prediction == 0 else 'Gaya Hidup'

    return category

# Example of using the function with user input
user_input = input("Masukkan isi berita: ")
predicted_category = predict_category(user_input)
print(f'Kategori berita: {predicted_category}')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Masukkan isi berita: Ia menyatakan bahwa penyediaan layanan online tersebut, dilakukan untuk meningkatkan kualitas layanan perizinan di Kota Surabaya melalui penyederhanaan alur, persyaratan, dan waktu sehingga memudahkan pelaku usaha untuk mengurus perizinan.  "Untuk memudahkan koordinasi semua petugas pemroses perizinan, Pemkot Surabaya juga telah melakukan digitalisasi proses perizinan dimana semua layanan perizinan dilaksanakan melalui aplikasi OSS RBA (Online Single Submission Risk Based Approach) dan SSW," ujarnya.  Di samping itu, Lasidi juga menyebutkan bahwa semua layanan perizinan dimonitor langsung oleh Wali Kota Surabaya melalui dashboard dan tidak boleh ada perizinan yang terlambat.  Selain kebijakan perizinan, Pemkot juga terus berupaya membuka destinasi wisata baru agar menarik investor membuka usaha baru di Kota Surabaya sesuai potensi wilayah.  "Pemkot Surabaya terus bersinergi dengan berbagai daerah dan instansi agar peran Kota Pahlawan sebagai citihub  perdagangan da